<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con el consumo eléctrico.** Las provincias con un PIB más alto o en crecimiento experimentan un mayor consumo de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Transformación

En este laboratorio, tu objetivo será limpiar y preparar los datos extraídos previamente de diferentes fuentes para su posterior análisis. Trabajarás con datos provenientes de la API de Red Eléctrica Española (REE) y del Instituto Nacional de Estadística (INE). Estos datos incluyen información sobre demanda y generación eléctrica a nivel provincial, así como datos demográficos y económicos. 


- Cargar los Datos Extraídos:

  - **Demanda Eléctrica:** Carga los datos de demanda eléctrica extraídos de la API de REE.

  - **Generación Eléctrica:** Carga los datos de generación eléctrica diferenciados por tipo de energía (eólica, solar, hidroeléctrica, etc.) a nivel provincial.

  - **Datos Demográficos:** Carga los datos demográficos por provincia extraídos de la web del INE.

  - **Datos Económicos:** Carga los datos del PIB por provincia obtenidos del INE.


-Limpieza de Datos:

- Datos de la API de REE:

  - **Demanda Eléctrica:**

    - **Conversión de Timestamps:** Asegúrate de que las fechas estén correctamente formateadas en `datetime`. Si es necesario, convierte los datos a un formato uniforme (por ejemplo, `YYYY-MM` para datos mensuales).

    - **Tratamiento de Valores Nulos:** Identifica y maneja los valores nulos en caso de que los haya. Puedes optar por eliminar filas con valores faltantes.

    - **Estandarización de Nombres de Provincias:** Verifica que los nombres de las provincias estén estandarizados y coincidan en todos los conjuntos de datos. Si hay inconsistencias, corrígelas.

  - **Generación Eléctrica:**

    - **Desagregación de Tecnologías:** Asegúrate de que los datos estén correctamente desglosados por tipo de energía. Revisa que los campos correspondientes a energía eólica, solar, hidroeléctrica, etc., estén bien identificados y sin errores.

    - **Normalización de Unidades:** Verifica que las unidades de energía estén estandarizadas (por ejemplo, MWh). Realiza las conversiones necesarias si se encuentran en otras unidades.

    - **Identificación de Outliers:** Revisa los valores extremos o atípicos en la generación de energía y decide si deben ser tratados o eliminados.

- Datos del INE:

  - **Datos Demográficos:**

    - **Consistencia en la Codificación de Provincias:** Asegúrate de que los nombres de las provincias en los datos demográficos coincidan con los nombres utilizados en los datos eléctricos.

    - **Revisión de Categorías:** Verifica que las categorías de edad, sexo, y nacionalidad estén correctamente etiquetadas y sean consistentes en todo el dataset.

    - **Manejo de Valores Faltantes:** Revisa la presencia de valores faltantes y decide cómo tratarlos (relleno, eliminación o sustitución).

  - **Datos Económicos:**

    - **Normalización del PIB:** Si los datos del PIB están en diferentes unidades o escalas, asegúrate de normalizarlos para que sean comparables entre provincias.

    - **Agrupación Temporal:** Si los datos económicos están disponibles en diferentes periodos temporales, agrúpalos y normalízalos para que coincidan con los datos eléctricos en términos de granularidad temporal (mensual o anual).

NOTA: Ten en cuenta que los datos los vamos a tener que insertar en una base de datos mañana, por lo que toda esta limpieza os recomendamos que la penséis para poder crear e insertar los datos mañana. 

In [303]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import Chrome


from time import sleep
# Selenium para establecer la configuración del driver
# -----------------------------------------------------------------------
from selenium import webdriver

# Para generar una barra de proceso en los bucles for
# -----------------------------------------------------------------------
from tqdm import tqdm

# Para trabajar con ficheros
# -----------------------------------------------------------------------
import os

import re

import zipfile
import shutil

import pandas as pd
pd.options.display.float_format = '{:20,.2f}'.format

import requests
import json

## Red Eléctrica Generación

In [304]:
years = [2019, 2020, 2021]

cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

cod_energias = {'Solar fotovoltaica':1,
                'Residuos renovables':2,
                'Generación renovable':3,
                'Hidráulica':4,
                'Eólica':5,
                'Solar térmica':6,
                'Otras renovables':7,
                'Hidroeólica':8}

In [305]:
df_completo_generacion = pd.DataFrame()

for year in years:
    for comunidad, id in cod_comunidades.items():
        try:
            archivo_path = f"/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/generacion/{year}/{comunidad}.json"
            #print(archivo_path)
            with open(archivo_path, 'r', encoding="utf-8") as archivo:
                datos = json.load(archivo)
            
            lista_datos = []

            for entry in datos["included"]:
                tipo = entry["type"]
                id_tipo = entry["id"]
                #descripcion = entry["attributes"].get("description")
                #actualizacion = entry["attributes"].get("last-update")
                
                for value_entry in entry["attributes"]["values"]:
                    lista_datos.append({
                        "ID Comunidad": id,
                        "Tipo": tipo,
                        #"ID": id_tipo,
                        #"Descripción": descripcion,
                        #"Última actualización": actualizacion,
                        "Mes": value_entry["datetime"],
                        "Año":year,
                        "Valor": value_entry["value"],
                        "Porcentaje": value_entry["percentage"]
                    })
            
            df = pd.DataFrame(lista_datos)
            
            df_completo_generacion = pd.concat([df_completo_generacion, df], ignore_index=True)
        except:
            pass

In [306]:
df_completo_generacion[["Mes","Resto"]] = df_completo_generacion["Mes"].str.split("T", expand = True)
df_completo_generacion.drop(columns="Resto", inplace=True)
df_completo_generacion["Mes"] = pd.to_datetime(df_completo_generacion["Mes"], format='%Y-%m-%d').dt.month

In [307]:
df_completo_generacion

,ID Comunidad,Tipo,Mes,Año,Valor,Porcentaje
0,8745,Solar fotovoltaica,1,2019,5.05,0.01
1,8745,Solar fotovoltaica,2,2019,5.53,0.01
2,8745,Solar fotovoltaica,3,2019,6.56,0.01
3,8745,Solar fotovoltaica,4,2019,7.92,0.02
4,8745,Solar fotovoltaica,5,2019,8.68,0.02
...,...,...,...,...,...,...
3552,20,Generación renovable,8,2021,"82,996.98",1.00
3553,20,Generación renovable,9,2021,"58,651.08",1.00
3554,20,Generación renovable,10,2021,"72,563.02",1.00
3555,20,Generación renovable,11,2021,"161,015.09",1.00


In [308]:
df_completo_generacion.dtypes

ID Comunidad      int64
Tipo             object
Mes               int32
Año               int64
Valor           float64
Porcentaje      float64
dtype: object

In [309]:
df_completo_generacion.isna().value_counts()

ID Comunidad  Tipo   Mes    Año    Valor  Porcentaje
False         False  False  False  False  False         3557
Name: count, dtype: int64

In [310]:
# Eliminamos porque hace referencia al total de energías renovables
df_completo_generacion = df_completo_generacion[df_completo_generacion['Tipo'] != 'Generación renovable']


In [311]:
df_tipos_energia = pd.DataFrame([df_completo_generacion["Tipo"].unique()])
df_tipos_energia = df_tipos_energia.T.reset_index()
df_tipos_energia.rename(columns={"index": "ID Energía", 0: "Tipo Energía"}, inplace=True)
df_tipos_energia

,ID Energía,Tipo Energía
0,0,Solar fotovoltaica
1,1,Residuos renovables
2,2,Hidráulica
3,3,Eólica
4,4,Solar térmica
5,5,Otras renovables
6,6,Hidroeólica


In [312]:
df_completo_generacion

,ID Comunidad,Tipo,Mes,Año,Valor,Porcentaje
0,8745,Solar fotovoltaica,1,2019,5.05,0.01
1,8745,Solar fotovoltaica,2,2019,5.53,0.01
2,8745,Solar fotovoltaica,3,2019,6.56,0.01
3,8745,Solar fotovoltaica,4,2019,7.92,0.02
4,8745,Solar fotovoltaica,5,2019,8.68,0.02
...,...,...,...,...,...,...
3540,20,Otras renovables,8,2021,645.33,0.01
3541,20,Otras renovables,9,2021,695.54,0.01
3542,20,Otras renovables,10,2021,588.32,0.01
3543,20,Otras renovables,11,2021,434.94,0.00


In [314]:
df_completo_generacion = df_completo_generacion.merge(df_tipos_energia, left_on="Tipo", right_on="Tipo Energía")
df_completo_generacion = df_completo_generacion.drop(columns = ["Tipo","Tipo Energía"])

In [315]:
df_completo_generacion = df_completo_generacion.reindex(columns = ["Año", "Mes", "ID Comunidad", "ID Energía", "Valor","Porcentaje"])

In [316]:
df_completo_generacion

,Año,Mes,ID Comunidad,ID Energía,Valor,Porcentaje
0,2019,1,8745,0,5.05,0.01
1,2019,2,8745,0,5.53,0.01
2,2019,3,8745,0,6.56,0.01
3,2019,4,8745,0,7.92,0.02
4,2019,5,8745,0,8.68,0.02
...,...,...,...,...,...,...
2904,2021,8,20,5,645.33,0.01
2905,2021,9,20,5,695.54,0.01
2906,2021,10,20,5,588.32,0.01
2907,2021,11,20,5,434.94,0.00


### Función outliers

In [399]:
def identificar_outliers(df, columna):
    """
    Identifica outliers en una columna de un DataFrame utilizando el método IQR.
    
    Parámetros:
    df (DataFrame): El DataFrame que contiene la columna a evaluar.
    columna (str): El nombre de la columna a evaluar.

    Retorna:
    DataFrame: Un DataFrame que contiene solo los outliers.
    """
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[columna] < lower_bound) | (df[columna] > upper_bound)]
    return outliers


## Comunidades

In [317]:
df_comunidades = pd.DataFrame([cod_comunidades]).T.reset_index()
df_comunidades.rename(columns={"index": "Comunidad", 0: "ID Comunidad"}, inplace=True)
df_comunidades = df_comunidades.reindex(columns=["ID Comunidad", "Comunidad"])
df_comunidades

,ID Comunidad,Comunidad
0,8744,Ceuta
1,8745,Melilla
2,4,Andalucía
3,5,Aragón
4,6,Cantabria
5,7,Castilla - La Mancha
6,8,Castilla y León
7,9,Cataluña
8,10,País Vasco
9,11,Principado de Asturias


## Provincias

In [386]:
provincias_comunidades = {
    'Albacete': 7,  # Castilla - La Mancha
    'Alicante/Alacant': 15,  # Comunitat Valenciana
    'Almería': 4,  # Andalucía
    'Araba/Álava': 10,  # País Vasco
    'Asturias': 11,  # Principado de Asturias
    'Ávila': 8,  # Castilla y León
    'Badajoz': 16,  # Extremadura
    'Balears,': 8743,  # Illes Balears
    'Barcelona': 9,  # Cataluña
    'Bizkaia': 10,  # País Vasco
    'Burgos': 8,  # Castilla y León
    'Cáceres': 16,  # Extremadura
    'Cádiz': 4,  # Andalucía
    'Cantabria': 6,  # Cantabria
    'Castellón/Castelló': 15,  # Comunitat Valenciana
    'Ciudad': 7,  # Castilla - La Mancha (Ciudad Real)
    'Córdoba': 4,  # Andalucía
    'Coruña,': 17,  # Galicia
    'Cuenca': 7,  # Castilla - La Mancha
    'Gipuzkoa': 10,  # País Vasco
    'Girona': 9,  # Cataluña
    'Granada': 4,  # Andalucía
    'Guadalajara': 7,  # Castilla - La Mancha
    'Huelva': 4,  # Andalucía
    'Huesca': 5,  # Aragón
    'Jaén': 4,  # Andalucía
    'León': 8,  # Castilla y León
    'Lleida': 9,  # Cataluña
    'Lugo': 17,  # Galicia
    'Madrid': 13,  # Comunidad de Madrid
    'Málaga': 4,  # Andalucía
    'Murcia': 21,  # Región de Murcia
    'Navarra': 14,  # Comunidad Foral de Navarra
    'Ourense': 17,  # Galicia
    'Palencia': 8,  # Castilla y León
    'Palmas,': 8742,  # Canarias (Las Palmas)
    'Pontevedra': 17,  # Galicia
    'Rioja,': 20,  # La Rioja
    'Salamanca': 8,  # Castilla y León
    'Santa Cruz de Tenerife': 8742,  # Canarias (Santa Cruz de Tenerife)
    'Segovia': 8,  # Castilla y León
    'Sevilla': 4,  # Andalucía
    'Soria': 8,  # Castilla y León
    'Tarragona': 9,  # Cataluña
    'Teruel': 5,  # Aragón
    'Toledo': 7,  # Castilla - La Mancha
    'Valencia/València': 15,  # Comunitat Valenciana
    'Valladolid': 8,  # Castilla y León
    'Zamora': 8,  # Castilla y León
    'Zaragoza': 5,  # Aragón
    'Ceuta': 8744,  # Ceuta
    'Melilla': 8745   # Melilla
}

In [392]:
df_provincias = pd.DataFrame([provincias_comunidades]).T.reset_index()
df_provincias.rename(columns = {"index":"Provincia",0:"ID Comunidad"})
df_provincias

,index,0
0,Albacete,7
1,Alicante/Alacant,15
2,Almería,4
3,Araba/Álava,10
4,Asturias,11
5,Ávila,8
6,Badajoz,16
7,"Balears,",8743
8,Barcelona,9
9,Bizkaia,10


## Red Eléctrica Demanda

In [318]:
df_completo_demanda = pd.DataFrame()

for year in years:
    for comunidad, id in cod_comunidades.items():
        try:
            archivo_path = f"/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/demanda/{year}/{comunidad}.json"
            #print(archivo_path)
            with open(archivo_path, 'r', encoding="utf-8") as archivo:
                datos = json.load(archivo)
            
            lista_datos = []

            for entry in datos["included"]:
                
                for value_entry in entry["attributes"]["values"]:
                    lista_datos.append({
                        "ID Comunidad": id,
                        "Mes": value_entry["datetime"],
                        "Año":year,
                        "Valor": value_entry["value"],
                    })
            
            df = pd.DataFrame(lista_datos)
            
            df_completo_demanda = pd.concat([df_completo_demanda, df], ignore_index=True)
        except:
            pass

In [319]:
df_completo_demanda[["Mes","Resto"]] = df_completo_demanda["Mes"].str.split("T", expand = True)
df_completo_demanda.drop(columns="Resto", inplace=True)
df_completo_demanda["Mes"] = pd.to_datetime(df_completo_demanda["Mes"], format='%Y-%m-%d').dt.month

In [320]:
df_completo_demanda = df_completo_demanda.reindex(columns=["Año","Mes","ID Comunidad","Valor"])
df_completo_demanda

,Año,Mes,ID Comunidad,Valor
0,2019,1,8744,"24,566,154.55"
1,2019,2,8744,"21,280,529.20"
2,2019,3,8744,"21,935,545.64"
3,2019,4,8744,"20,692,209.65"
4,2019,5,8744,"21,134,292.13"
...,...,...,...,...
679,2021,8,20,"22,059,793.28"
680,2021,9,20,"20,972,956.06"
681,2021,10,20,"20,197,477.97"
682,2021,11,20,"21,433,492.69"


In [321]:
df_completo_demanda.isna().value_counts()

Año    Mes    ID Comunidad  Valor
False  False  False         False    684
Name: count, dtype: int64

In [344]:
df_suma = df_completo_generacion[((df_completo_generacion["Año"] == 2019) & df_completo_generacion["Mes"] == 1) & (df_completo_generacion["ID Comunidad"] == 8745) & (df_completo_generacion["ID Energía"] == 0)]["Valor"].sum()
df_suma

np.float64(40.05)

In [338]:
df_nuevo_porc = df_completo_generacion[((df_completo_generacion["Año"] == 2019) & df_completo_generacion["Mes"] == 1) & (df_completo_generacion["ID Comunidad"] == 8745) & (df_completo_generacion["ID Energía"] == 0)]["Valor"]/df_suma

In [339]:
df_nuevo_porc

0                    0.13
2                    0.16
4                    0.22
6                    0.20
8                    0.17
10                   0.12
Name: Valor, dtype: float64

In [361]:
#df_suma = df_completo_generacion[((df_completo_generacion["Año"] == {year}) & df_completo_generacion["Mes"] == {month}) & (df_completo_generacion["ID Comunidad"] == {id}) & (df_completo_generacion["ID Energía"] == {tipo})]["Valor"].sum()

#df_nuevo_porc = df_completo_generacion[((df_completo_generacion["Año"] == {year}) & df_completo_generacion["Mes"] == {month}) & (df_completo_generacion["ID Comunidad"] == {id}) & (df_completo_generacion["ID Energía"] == {tipo})]["Valor"]/df_suma

In [357]:
for year in years:
    for month in range(1, 13):  # Assuming months should be 1 to 12
        for id, comunidad in cod_comunidades.items():  # Assuming cod_comunidades is a dictionary
            for tipo in range(0, 7):
                df_suma = df_completo_generacion[
                    (df_completo_generacion["Año"] == year) &
                    (df_completo_generacion["Mes"] == month) &
                    (df_completo_generacion["ID Comunidad"] == id) &
                    (df_completo_generacion["ID Energía"] == tipo)
                ]["Valor"].sum()
                
                df_completo_generacion.loc[
                    (df_completo_generacion["Año"] == year) &
                    (df_completo_generacion["Mes"] == month) &
                    (df_completo_generacion["ID Comunidad"] == id) &
                    (df_completo_generacion["ID Energía"] == tipo), "Nuevo Porcentaje"
                ] = round(df_completo_generacion[
                    (df_completo_generacion["Año"] == year) &
                    (df_completo_generacion["Mes"] == month) &
                    (df_completo_generacion["ID Comunidad"] == id) &
                    (df_completo_generacion["ID Energía"] == tipo)
                ]["Valor"] / df_suma, 5)


In [359]:
df_completo_generacion

,Año,Mes,ID Comunidad,ID Energía,Valor,Porcentaje,Nuevo Porcentaje
0,2019,1,8745,0,5.05,0.01,0.00
1,2019,2,8745,0,5.53,0.01,0.00
2,2019,3,8745,0,6.56,0.01,0.00
3,2019,4,8745,0,7.92,0.02,0.00
4,2019,5,8745,0,8.68,0.02,0.00
...,...,...,...,...,...,...,...
2904,2021,8,20,5,645.33,0.01,0.00
2905,2021,9,20,5,695.54,0.01,0.00
2906,2021,10,20,5,588.32,0.01,0.00
2907,2021,11,20,5,434.94,0.00,0.00


## IPC Población

In [366]:
df_poblacion = pd.read_csv("/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/DatosDescargados/poblacion.csv", sep=';', encoding='utf-8')

In [370]:
df_poblacion

,Provincias,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total
0,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Ambos sexos,2022,47.475.420
1,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Ambos sexos,2021,47.385.107
2,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Ambos sexos,2020,47.450.795
3,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Ambos sexos,2019,47.026.208
4,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Ambos sexos,2018,46.722.980
...,...,...,...,...,...,...
63595,52 Melilla,65 y más,% Extranjeros,Mujeres,2002,"5,13"
63596,52 Melilla,65 y más,% Extranjeros,Mujeres,2001,"4,65"
63597,52 Melilla,65 y más,% Extranjeros,Mujeres,2000,"3,34"
63598,52 Melilla,65 y más,% Extranjeros,Mujeres,1999,"0,81"


In [419]:
sucio = df_poblacion[
    (df_poblacion["Edad (3 grupos de edad)"] != "TOTAL EDADES") & 
    (df_poblacion["Españoles/Extranjeros"] != "TOTAL") & 
    (df_poblacion["Provincias"] != "TOTAL ESPAÑA") &
    (df_poblacion["Españoles/Extranjeros"] != "% Extranjeros") &
    (df_poblacion["Sexo"] != "Ambos sexos")
]

In [429]:
sucio['Provincias'] = sucio['Provincias'].apply(lambda x: "".join(re.findall(pattern= r"\d+", string=x))).astype(int)

/var/folders/yj/y1bv8zk12j11bnjq1hc6x_d80000gn/T/ipykernel_52457/1966336634.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sucio['Provincias'] = sucio['Provincias'].apply(lambda x: "".join(re.findall(pattern= r"\d+", string=x))).astype(int)


In [421]:
sucio.reset_index(inplace=True, drop=True)

In [430]:
sucio.head(1)

,Provincias,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total
0,2,0-15 años,Españoles,Hombres,2022,26.845


In [395]:
provincias = sucio["Provincias"].unique()
codigos = dict()
for provincia in provincias:
    splits = provincia.split()
    codigos["".join(re.findall(pattern= r" (\D+)", string=provincia))] = splits[0]

codigos

{'Albacete': '02',
 'Alicante/Alacant': '03',
 'Almería': '04',
 'Araba/Álava': '01',
 'Asturias': '33',
 'Ávila': '05',
 'Badajoz': '06',
 'Balears, Illes': '07',
 'Barcelona': '08',
 'Bizkaia': '48',
 'Burgos': '09',
 'Cáceres': '10',
 'Cádiz': '11',
 'Cantabria': '39',
 'Castellón/Castelló': '12',
 'Ciudad Real': '13',
 'Córdoba': '14',
 'Coruña, A': '15',
 'Cuenca': '16',
 'Gipuzkoa': '20',
 'Girona': '17',
 'Granada': '18',
 'Guadalajara': '19',
 'Huelva': '21',
 'Huesca': '22',
 'Jaén': '23',
 'León': '24',
 'Lleida': '25',
 'Lugo': '27',
 'Madrid': '28',
 'Málaga': '29',
 'Murcia': '30',
 'Navarra': '31',
 'Ourense': '32',
 'Palencia': '34',
 'Palmas, Las': '35',
 'Pontevedra': '36',
 'Rioja, La': '26',
 'Salamanca': '37',
 'Santa Cruz de Tenerife': '38',
 'Segovia': '40',
 'Sevilla': '41',
 'Soria': '42',
 'Tarragona': '43',
 'Teruel': '44',
 'Toledo': '45',
 'Valencia/València': '46',
 'Valladolid': '47',
 'Zamora': '49',
 'Zaragoza': '50',
 'Ceuta': '51',
 'Melilla': '52'}

In [428]:
lista_tuplas_provincias = []
for k, v in codigos.items():
    lista_tuplas_provincias.append(tuple([int(v),k]))

lista_tuplas_provincias

[(2, 'Albacete'),
 (3, 'Alicante/Alacant'),
 (4, 'Almería'),
 (1, 'Araba/Álava'),
 (33, 'Asturias'),
 (5, 'Ávila'),
 (6, 'Badajoz'),
 (7, 'Balears, Illes'),
 (8, 'Barcelona'),
 (48, 'Bizkaia'),
 (9, 'Burgos'),
 (10, 'Cáceres'),
 (11, 'Cádiz'),
 (39, 'Cantabria'),
 (12, 'Castellón/Castelló'),
 (13, 'Ciudad Real'),
 (14, 'Córdoba'),
 (15, 'Coruña, A'),
 (16, 'Cuenca'),
 (20, 'Gipuzkoa'),
 (17, 'Girona'),
 (18, 'Granada'),
 (19, 'Guadalajara'),
 (21, 'Huelva'),
 (22, 'Huesca'),
 (23, 'Jaén'),
 (24, 'León'),
 (25, 'Lleida'),
 (27, 'Lugo'),
 (28, 'Madrid'),
 (29, 'Málaga'),
 (30, 'Murcia'),
 (31, 'Navarra'),
 (32, 'Ourense'),
 (34, 'Palencia'),
 (35, 'Palmas, Las'),
 (36, 'Pontevedra'),
 (26, 'Rioja, La'),
 (37, 'Salamanca'),
 (38, 'Santa Cruz de Tenerife'),
 (40, 'Segovia'),
 (41, 'Sevilla'),
 (42, 'Soria'),
 (43, 'Tarragona'),
 (44, 'Teruel'),
 (45, 'Toledo'),
 (46, 'Valencia/València'),
 (47, 'Valladolid'),
 (49, 'Zamora'),
 (50, 'Zaragoza'),
 (51, 'Ceuta'),
 (52, 'Melilla')]

## INE PIB

In [368]:
df_pib = pd.read_csv("/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/DatosDescargados/pib.csv", sep=';', encoding='latin-1')

In [406]:
df_pib

,Provincias,Ramas de actividad,periodo,Total
0,02 Albacete,PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO,2021(P),8.663.814
1,02 Albacete,PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO,2020,7.855.556
2,02 Albacete,PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO,2019,8.475.070
3,02 Albacete,"A. Agricultura, ganadería, silvicultura y pesca",2021(P),884.324
4,02 Albacete,"A. Agricultura, ganadería, silvicultura y pesca",2020,768.976
...,...,...,...,...
1711,52 Melilla,Impuestos netos sobre los productos,2020,132.490
1712,52 Melilla,Impuestos netos sobre los productos,2019,151.213
1713,52 Melilla,PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO,2021(P),1.616.543
1714,52 Melilla,PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO,2020,1.514.009


In [407]:
sucio2 = df_pib[
    (df_pib["Ramas de actividad"] != "PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO")
]

sucio2[["Año","Resto"]] = sucio2["periodo"].str.split("(", expand = True)
sucio2 = sucio2.drop(columns = ["periodo", "Resto"])

/var/folders/yj/y1bv8zk12j11bnjq1hc6x_d80000gn/T/ipykernel_52457/3596973143.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sucio2[["Año","Resto"]] = sucio2["periodo"].str.split("(", expand = True)
/var/folders/yj/y1bv8zk12j11bnjq1hc6x_d80000gn/T/ipykernel_52457/3596973143.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sucio2[["Año","Resto"]] = sucio2["periodo"].str.split("(", expand = True)


In [408]:
sucio2

,Provincias,Ramas de actividad,Total,Año
3,02 Albacete,"A. Agricultura, ganadería, silvicultura y pesca",884.324,2021
4,02 Albacete,"A. Agricultura, ganadería, silvicultura y pesca",768.976,2020
5,02 Albacete,"A. Agricultura, ganadería, silvicultura y pesca",791.464,2019
6,02 Albacete,"B_E. Industrias extractivas, industria manufac...",1.397.006,2021
7,02 Albacete,"B_E. Industrias extractivas, industria manufac...",1.194.438,2020
...,...,...,...,...
1708,52 Melilla,Valor añadido bruto total,1.381.519,2020
1709,52 Melilla,Valor añadido bruto total,1.473.873,2019
1710,52 Melilla,Impuestos netos sobre los productos,153.994,2021
1711,52 Melilla,Impuestos netos sobre los productos,132.490,2020


In [431]:
sucio2['Provincias'] = sucio2['Provincias'].apply(lambda x: "".join(re.findall(pattern= r"\d+", string=x))).astype(int)

In [433]:
sucio2.reset_index(inplace=True,drop=True)

In [435]:
sucio2 = sucio2.reindex(columns= ["Provincias", "Ramas de actividad", "Año", "Total"])

In [436]:
sucio2

,Provincias,Ramas de actividad,Año,Total
0,2,"A. Agricultura, ganadería, silvicultura y pesca",2021,884.324
1,2,"A. Agricultura, ganadería, silvicultura y pesca",2020,768.976
2,2,"A. Agricultura, ganadería, silvicultura y pesca",2019,791.464
3,2,"B_E. Industrias extractivas, industria manufac...",2021,1.397.006
4,2,"B_E. Industrias extractivas, industria manufac...",2020,1.194.438
...,...,...,...,...
1399,52,Valor añadido bruto total,2020,1.381.519
1400,52,Valor añadido bruto total,2019,1.473.873
1401,52,Impuestos netos sobre los productos,2021,153.994
1402,52,Impuestos netos sobre los productos,2020,132.490
